In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import milp, LinearConstraint, Bounds

In [4]:
df = pd.read_csv("../input/KPD_pool.csv")

In [5]:
bt_D = pd.get_dummies(data=df["DonorBloodType"],prefix="bt_D",dtype=int)
bt_P = pd.get_dummies(data=df["PatientBloodType"],prefix="bt_P",dtype=int)

bt_D = bt_D[['bt_D_AB', 'bt_D_A', 'bt_D_B', 'bt_D_O']]
bt_P = bt_P[['bt_P_AB', 'bt_P_A', 'bt_P_B', 'bt_P_O']]

bt_D = np.asarray(bt_D,dtype=int)
bt_P = np.asarray(bt_P,dtype=int)

t_D = pd.get_dummies(data=df["DonorTissueType"],prefix="t_D",dtype=int)
t_P = pd.get_dummies(data=df["PatientTissueType"],prefix="t_P",dtype=int)

t_D = np.asarray(t_D,dtype=int)
t_P = np.asarray(t_P,dtype=int)

In [6]:
M_tissue = np.identity(n=5,dtype=int)
M_blood = np.array([[1,0,0,0],
           [1,1,0,0],
           [1,0,1,0],
           [1,1,1,1]], dtype=int)

In [7]:
pairs = (bt_D @ M_blood @ bt_P.T) * (t_D @ M_tissue @ t_P.T)

# constraints

In [8]:
coefficients = -pairs.flatten()

In [9]:
D = pairs.shape[0]
P = pairs.shape[1]

In [ ]:
A_donor = np.zeros(shape=(D, D*P), dtype=int)
A_patient = np.zeros(shape=(P, D*P), dtype=int)

for i in range(D):
    for j in range(P):
        A_donor[i, i * P + j] = 1

for j in range(P):
    for i in range(D):
        A_patient[j, i * P + j] = 1

A = np.vstack((A_donor, A_patient))
b_ub = np.ones(D + P)

In [11]:
constraints = LinearConstraint(
    A=A,
    lb=0,
    ub=b_ub
)

In [12]:
bounds = Bounds(
    lb=0,
    ub=1
)

In [13]:
integrality = np.ones(D*P, dtype=int)

In [14]:
result_milp = milp(
    c=coefficients,
    bounds=bounds,
    constraints=constraints,
    integrality=integrality
)

In [15]:
result_milp

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -16.0
              x: [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
 mip_node_count: 1
 mip_dual_bound: -16.0
        mip_gap: 0.0

In [16]:
type(result_milp.x)

numpy.ndarray

In [17]:
result_selection_matrix = result_milp.x.reshape(D, P)

In [18]:
result_selection_matrix.shape

(25, 25)

In [19]:
n_donations = 0
for i in range(D):
    for j in range(P):
        if result_selection_matrix[i, j] == 1:
            print(f"Donor {i+1} --> Patient {j+1}")
            n_donations = n_donations + 1

print(f"number of donations = {n_donations}")

Donor 1 --> Patient 2
Donor 2 --> Patient 12
Donor 5 --> Patient 10
Donor 9 --> Patient 22
Donor 10 --> Patient 15
Donor 11 --> Patient 24
Donor 14 --> Patient 6
Donor 15 --> Patient 7
Donor 16 --> Patient 17
Donor 17 --> Patient 21
Donor 18 --> Patient 13
Donor 19 --> Patient 23
Donor 20 --> Patient 11
Donor 22 --> Patient 4
Donor 23 --> Patient 1
Donor 24 --> Patient 14
number of donations = 16
